In [1]:
import pandas
import altair as alt
import numpy
import scipy
import scipy.stats as stats

In [2]:

import pickle

with open('data.pickle', 'rb') as f:
    data = pickle.load(f, encoding='binary')

data.head(5)


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,OCC,OCC_END
0,506547392,03/29/2018,20:30:00,<NA>,NaN,32.0,2018-03-30,351,CRIMINAL MISCHIEF & RELATED OF,254.0,...,40.810877,-73.941064,"(40.810877241, -73.941064151)",PATROL BORO MAN NORTH,NaN,25-44,WHITE,F,2018-03-29,<NA>
1,629632833,02/06/2018,23:15:00,<NA>,NaN,52.0,2018-02-07,341,PETIT LARCENY,333.0,...,40.873671,-73.908014,"(40.873671035, -73.908013649)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,D,2018-02-06,<NA>
2,787203902,11/21/2018,00:15:00,11/21/2018,00:20:00,75.0,2018-11-21,341,PETIT LARCENY,321.0,...,40.651782,-73.885457,"(40.651782232, -73.885456761)",PATROL BORO BKLYN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-11-21,11/21/2018
3,280364018,06/09/2018,21:42:00,06/09/2018,21:43:00,10.0,2018-06-10,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,...,40.759310,-73.994706,"(40.759310399, -73.994706072)",PATROL BORO MAN SOUTH,NaN,18-24,WHITE HISPANIC,F,2018-06-09,06/09/2018
4,985800320,11/10/2018,19:40:00,11/10/2018,19:45:00,19.0,2018-11-10,341,PETIT LARCENY,333.0,...,40.764536,-73.970728,"(40.764535539, -73.970728388)",PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-11-10,11/10/2018


In [3]:
def make_categorical(dataframe, column):
    # make the column of the dataframe categorical
    dataframe[column] = dataframe[column].astype('category')

# get a lst of all the columns in the dataframe
columns = ['ADDR_PCT_CD', 'BORO_NM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT', 'HOUSING_PSA', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 'PD_CD', 'PD_DESC', 'STATION_NAME', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']

    # make each column categorical
for column in columns:
    make_categorical(data, column)


In [4]:
vic_sex_count = data['VIC_SEX'].value_counts()

# using altair plot the number of crimes per SUSPECT SEX in a bar chart
alt.Chart(vic_sex_count.reset_index()).mark_bar().encode(
    x = alt.X('index', axis= alt.Axis(title = 'Victim Sex')),
    y = alt.Y('VIC_SEX',axis= alt.Axis(format='e', title = 'Number of Complaints'))
).properties(
    title='Number of complaints per victim sex',
    width=1000,
    height=500
)

alt.Chart(...)

In [5]:
#make a percentage distribution in a table
vic_sex_counts = data.groupby('VIC_SEX').size().reset_index(name='complaint_count')
vic_sex_counts['complaint_percentage'] = 100 * vic_sex_counts['complaint_count'] / vic_sex_counts['complaint_count'].sum()
vic_sex_counts

,VIC_SEX,complaint_count,complaint_percentage
0,D,973080,12.477640
1,E,1187098,15.221957
2,F,3054751,39.170555
3,M,2583657,33.129797
4,U,4,0.000051


In [6]:
vic_race_count = data['VIC_RACE'].value_counts()
# using altair plot the number of crimes per SUSPECT RACE in a bar chart
alt.Chart(vic_race_count.reset_index()).mark_bar().encode(
    x = alt.X('index', axis= alt.Axis(title = 'Victim Race')),
    y = alt.Y('VIC_RACE',axis= alt.Axis(format='e', title = 'Number of Complaints'))    
).properties(
    title='Number of complaints per victim race',
    width=1000,
    height=500
)

alt.Chart(...)

In [7]:
#make a percentage distribution in a table
vic_race_counts = data.groupby('VIC_RACE').size().reset_index(name='complaint_count')
vic_race_counts['complaint_percentage'] = 100 * vic_race_counts['complaint_count'] / vic_race_counts['complaint_count'].sum()
vic_race_counts

,VIC_RACE,complaint_count,complaint_percentage
0,AMERICAN INDIAN/ALASKAN NATIVE,34811,0.446380
1,ASIAN / PACIFIC ISLANDER,462774,5.934135
2,BLACK,1881114,24.121460
3,BLACK HISPANIC,269075,3.450339
4,OTHER,30,0.000385
5,UNKNOWN,2553116,32.738519
6,WHITE,1329750,17.051339
7,WHITE HISPANIC,1267838,16.257443


In [8]:
#create a new column to remove unknown values
data['VIC_RACE2'] = numpy.where(data['VIC_RACE'].str.contains("UNKNOWN"), None, data['VIC_RACE'])


In [9]:
vic_race_count2 = data['VIC_RACE2'].value_counts()
# using altair plot the number of crimes per SUSPECT RACE in a bar chart without the unknown category
alt.Chart(vic_race_count2.reset_index()).mark_bar().encode(
    x = alt.X('index', axis= alt.Axis(title = 'Victim Race')),
    y = alt.Y('VIC_RACE2',axis= alt.Axis(format='e', title = 'Number of complaints'))    
).properties(
    title='Number of complaints per victim race',
    width=1000,
    height=500
)

alt.Chart(...)

In [10]:
#make a percentage distribution in a table without the unknown category
vic_race_counts2 = data.groupby('VIC_RACE2').size().reset_index(name='complaint_count')
vic_race_counts2['complaint_percentage'] = 100 * vic_race_counts['complaint_count'] / vic_race_counts['complaint_count'].sum()
vic_race_counts2

,VIC_RACE2,complaint_count,complaint_percentage
0,AMERICAN INDIAN/ALASKAN NATIVE,34811,0.446380
1,ASIAN / PACIFIC ISLANDER,462774,5.934135
2,BLACK,1881114,24.121460
3,BLACK HISPANIC,269075,3.450339
4,OTHER,30,0.000385
5,WHITE,1329750,32.738519
6,WHITE HISPANIC,1267838,17.051339


In [11]:
#where there is NaN, fill it with unknown
data['VIC_AGE_GROUP'] = data['VIC_AGE_GROUP'].fillna('UNKNOWN')

In [12]:
#check whether the previous step worked
data['VIC_AGE_GROUP'].unique()

[25-44, UNKNOWN, 18-24, <18, 45-64, 65+]
Categories (6, object): [25-44, UNKNOWN, 18-24, <18, 45-64, 65+]

In [13]:
vic_age_count = data['VIC_AGE_GROUP'].value_counts()

categoryNames    = [ '<18', '18-24', '25-44', '45-64', '65+' ]

# using altair plot the number of crimes per VICTIM AGE GROUP in a bar chart
alt.Chart(vic_age_count.reset_index()).mark_bar().encode(
    x = alt.X('index', sort = categoryNames, axis= alt.Axis(title = 'Victim Age Group')),
    y = alt.Y('VIC_AGE_GROUP',axis= alt.Axis(format='e', title = 'Number of Complaints'))
).properties(
    title='Number of complaints per victim age group',
    width=1000,
    height=500
)

alt.Chart(...)

In [14]:
#make a percentage distribution in a table
vic_age_counts2 = data.groupby('VIC_AGE_GROUP').size().reset_index(name='complaint_count')
vic_age_counts2['complaint_percentage'] = 100 * vic_age_counts2['complaint_count'] / vic_age_counts2['complaint_count'].sum()
vic_age_counts2

,VIC_AGE_GROUP,complaint_count,complaint_percentage
0,18-24,786674,10.086989
1,25-44,2591295,33.226425
2,45-64,1353058,17.349349
3,65+,279818,3.587917
4,<18,351174,4.502867
5,UNKNOWN,2436879,31.246453


In [15]:
#create a new column where unknown values are removed
data['VIC_AGE_GROUP2'] = numpy.where(data['SUSP_AGE_GROUP'].str.contains("UNKNOWN"), None, data['SUSP_AGE_GROUP'])



In [16]:
vic_age_count2 = data['VIC_AGE_GROUP2'].value_counts()

categoryNames    = [ '<18', '18-24', '25-44', '45-64', '65+' ]

# using altair plot the number of crimes per VICTIM AGE GROUP in a bar chart without the unknown category
alt.Chart(vic_age_count2.reset_index()).mark_bar().encode(
    x = alt.X('index', sort = categoryNames, axis= alt.Axis(title = 'Victim Age Group')),
    y = alt.Y('VIC_AGE_GROUP2',axis= alt.Axis(format='e', title = 'Number of Complaints'))
).properties(
    title='Number of complaints per victim age group',
    width=1000,
    height=500
)

alt.Chart(...)

In [17]:
#make a percentage distribution in a table without unknown category
vic_age_counts = data.groupby('VIC_AGE_GROUP2').size().reset_index(name='complaint_count')
vic_age_counts['complaint_percentage'] = 100 * vic_age_counts['complaint_count'] / vic_age_counts['complaint_count'].sum()
vic_age_counts

,VIC_AGE_GROUP2,complaint_count,complaint_percentage
0,18-24,417534,19.941951
1,25-44,1127473,53.849534
2,45-64,399389,19.075323
3,65+,34127,1.629949
4,<18,115224,5.503244


In [21]:
#get year from occ
data['Year']=data['OCC'].dt.year 

In [22]:
#turn year into a string
data['Year'] = data['Year'].astype(str)

In [23]:
#group complaints per year
year_counts = data.groupby('Year').size().reset_index(name='complaint_count')

In [24]:
year_counts

,Year,complaint_count
0,2006,527816
1,2007,534640
2,2008,528619
3,2009,510936
4,2010,510092
5,2011,498542
6,2012,504656
7,2013,495789
8,2014,491988
9,2015,478567


In [25]:
#group so each year the distribution of complaints by victim sex is shown
year_counts_sex = data.groupby(['Year', 'VIC_SEX']).size().reset_index(name='complaint_count')

#turn year and complaint count into an integer
year_counts_sex['Year'] = year_counts_sex['Year'].astype(int)
year_counts_sex['complaint_count'] = year_counts_sex['complaint_count'].astype(int)

#turn victim sex into a string
year_counts_sex['VIC_SEX'] = year_counts_sex['VIC_SEX'].astype(str)


In [26]:
#plot a chart where the number of complaints per victim sex per year is shown
alt.Chart(year_counts_sex).mark_line().encode(
    alt.X('Year:O', axis = alt.Axis(title = 'Year')),
    alt.Y('complaint_count:Q', stack = "zero", axis = alt.Axis(title = 'Complaint Count')),
    alt.Color('VIC_SEX:O', scale=alt.Scale(scheme='lighttealblue'),title = 'Sex')
).properties(
    title='Number of complaints per victim sex per year',
    width = 1000,
    height = 500
)

alt.Chart(...)

In [27]:
#group so each year the distribution of complaints by victim race is shown
year_counts_race = data.groupby(['Year', 'VIC_RACE']).size().reset_index(name='complaint_count')

#turn year and complaint count into an integer
year_counts_race['Year'] = year_counts_race['Year'].astype(int)
year_counts_race['complaint_count'] = year_counts_race['complaint_count'].astype(int)

#turn victim race into a string
year_counts_race['VIC_RACE'] = year_counts_race['VIC_RACE'].astype(str)


In [28]:
#plot a chart where the number of complaints per victim race per year is shown
alt.Chart(year_counts_race).mark_line().encode(
    alt.X('Year:O', axis = alt.Axis(title = 'Year')),
    alt.Y('complaint_count:Q', stack = "zero", axis = alt.Axis(title = 'Complaint Count')),
    alt.Color('VIC_RACE:O', scale=alt.Scale(scheme='lighttealblue'),title = 'Race')
).properties(
    title='Number of complaints per victim race per year',
    width = 1000,
    height = 500
)

alt.Chart(...)

In [29]:
#group so each year the distribution of complaints by victim age group is shown
year_counts_age = data.groupby(['Year', 'VIC_AGE_GROUP']).size().reset_index(name='complaint_count')

#turn year and complaint count into an integer
year_counts_age['Year'] = year_counts_age['Year'].astype(int)
year_counts_age['complaint_count'] = year_counts_age['complaint_count'].astype(int)

#turn victim age group into a string
year_counts_age['VIC_AGE_GROUP'] = year_counts_age['VIC_AGE_GROUP'].astype(str)


In [30]:
#plot a chart where the number of complaints per victim age group per year is shown
alt.Chart(year_counts_age).mark_line().encode(
    alt.X('Year:O', axis = alt.Axis(title = 'Year')),
    alt.Y('complaint_count:Q', stack = "zero", axis = alt.Axis(title = 'Complaint Count')),
    alt.Color('VIC_AGE_GROUP:O', scale=alt.Scale(scheme='lighttealblue'),title = 'Age Group')
).properties(
    title='Number of complaints per age group per year',
    width = 1000,
    height = 500
)

alt.Chart(...)

In [31]:
#make a new dataset with only sex category U
data6 = data.loc[data['VIC_SEX'] == 'U']

In [32]:
#make a new dataset with only sex category D
data2 =data.loc[data['VIC_SEX'] == 'D']

In [33]:
#make a new dataset with only sex category E
data3 = data.loc[data['VIC_SEX'] == 'E']

In [34]:
data2

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,OCC,OCC_END,VIC_RACE2,VIC_AGE_GROUP2,Year
1,629632833,02/06/2018,23:15:00,<NA>,NaN,52.0,2018-02-07,341,PETIT LARCENY,333.0,...,PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,D,2018-02-06,<NA>,None,45-64,2018
2,787203902,11/21/2018,00:15:00,11/21/2018,00:20:00,75.0,2018-11-21,341,PETIT LARCENY,321.0,...,PATROL BORO BKLYN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-11-21,11/21/2018,None,25-44,2018
4,985800320,11/10/2018,19:40:00,11/10/2018,19:45:00,19.0,2018-11-10,341,PETIT LARCENY,333.0,...,PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-11-10,11/10/2018,None,<18,2018
7,683416529,09/12/2018,18:30:00,09/12/2018,18:35:00,46.0,2018-09-12,121,CRIMINAL MISCHIEF & RELATED OF,269.0,...,PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,D,2018-09-12,09/12/2018,None,25-44,2018
19,752125610,02/24/2018,18:24:00,02/24/2018,19:39:00,110.0,2018-02-24,341,PETIT LARCENY,333.0,...,PATROL BORO QUEENS NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-02-24,02/24/2018,None,None,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7825446,784316659,08/28/2018,07:50:00,08/28/2018,08:00:00,103.0,2018-08-28,341,PETIT LARCENY,333.0,...,PATROL BORO QUEENS SOUTH,NaN,UNKNOWN,UNKNOWN,D,2018-08-28,08/28/2018,None,25-44,2018
7825452,508147626,07/20/2018,14:10:00,07/20/2018,14:20:00,103.0,2018-07-20,341,PETIT LARCENY,333.0,...,PATROL BORO QUEENS SOUTH,NaN,UNKNOWN,UNKNOWN,D,2018-07-20,07/20/2018,None,45-64,2018
7825466,449013529,06/08/2018,07:55:00,06/08/2018,08:22:00,42.0,2018-06-08,341,PETIT LARCENY,333.0,...,PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,D,2018-06-08,06/08/2018,None,45-64,2018
7825479,224667280,02/01/2018,18:20:00,02/01/2018,18:25:00,90.0,2018-02-01,341,PETIT LARCENY,333.0,...,PATROL BORO BKLYN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-02-01,02/01/2018,None,45-64,2018


In [35]:
#count victim race of people with sex category D to see what gender they have
data2['VIC_RACE'].value_counts()

UNKNOWN                           965655
BLACK                               2162
WHITE                               2143
WHITE HISPANIC                      1652
ASIAN / PACIFIC ISLANDER            1059
BLACK HISPANIC                       275
AMERICAN INDIAN/ALASKAN NATIVE        96
OTHER                                  0
Name: VIC_RACE, dtype: int64

In [36]:
#count victim race of people with sex category E to see what gender they have
data3['VIC_RACE'].value_counts()

UNKNOWN                           1183709
BLACK                                1379
WHITE HISPANIC                        820
WHITE                                 776
BLACK HISPANIC                        215
ASIAN / PACIFIC ISLANDER              178
AMERICAN INDIAN/ALASKAN NATIVE         20
OTHER                                   0
Name: VIC_RACE, dtype: int64

In [37]:
#count victim race of people with sex category U to see what gender they have
data6['VIC_RACE'].value_counts()

UNKNOWN                           3
ASIAN / PACIFIC ISLANDER          1
WHITE HISPANIC                    0
WHITE                             0
OTHER                             0
BLACK HISPANIC                    0
BLACK                             0
AMERICAN INDIAN/ALASKAN NATIVE    0
Name: VIC_RACE, dtype: int64

In [38]:
#only select data in year 2010, as that is where the population census data is from. Done for the chi square test
demographic = data.loc[data['Year'] == '2010']


In [39]:
demographic

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,OCC,OCC_END,VIC_RACE2,VIC_AGE_GROUP2,Year
7663,712486868,06/04/2010,08:00:00,02/03/2018,14:00:00,114.0,2018-02-06,578,HARRASSMENT 2,638.0,...,PATROL BORO QUEENS NORTH,NaN,65+,WHITE HISPANIC,F,2010-06-04,02/03/2018,WHITE HISPANIC,45-64,2010
11931,997241474,02/14/2010,20:00:00,<NA>,NaN,9.0,2018-02-14,341,PETIT LARCENY,338.0,...,PATROL BORO MAN SOUTH,NaN,65+,BLACK,M,2010-02-14,<NA>,BLACK,None,2010
12361,527425056,01/01/2010,00:01:00,<NA>,NaN,24.0,2018-04-11,233,SEX CRIMES,175.0,...,PATROL BORO MAN NORTH,NaN,UNKNOWN,WHITE,F,2010-01-01,<NA>,WHITE,None,2010
14205,674476591,06/19/2010,23:00:00,04/10/2018,10:00:00,83.0,2018-04-12,340,FRAUDS,718.0,...,PATROL BORO BKLYN NORTH,NaN,25-44,BLACK HISPANIC,F,2010-06-19,04/10/2018,BLACK HISPANIC,None,2010
14420,269323272,12/01/2010,00:01:00,06/30/2011,23:59:00,90.0,2018-02-21,104,RAPE,157.0,...,PATROL BORO BKLYN NORTH,NaN,<18,WHITE HISPANIC,F,2010-12-01,06/30/2011,WHITE HISPANIC,None,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813636,394338898,12/14/2010,18:20:00,12/14/2018,18:30:00,105.0,2018-12-14,106,FELONY ASSAULT,109.0,...,PATROL BORO QUEENS SOUTH,NaN,25-44,ASIAN / PACIFIC ISLANDER,M,2010-12-14,12/14/2018,ASIAN / PACIFIC ISLANDER,25-44,2010
7814756,489264865,06/02/2010,20:00:00,<NA>,NaN,73.0,2018-06-02,116,SEX CRIMES,177.0,...,PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M,2010-06-02,<NA>,BLACK,None,2010
7819285,317559834,08/01/2010,00:01:00,04/01/2018,23:59:00,19.0,2018-05-03,109,GRAND LARCENY,422.0,...,PATROL BORO MAN NORTH,NaN,25-44,BLACK,M,2010-08-01,04/01/2018,BLACK,None,2010
7822247,211434671,01/01/2010,00:00:00,12/31/2013,23:55:00,72.0,2018-03-08,112,THEFT-FRAUD,739.0,...,PATROL BORO BKLYN SOUTH,NaN,25-44,WHITE HISPANIC,F,2010-01-01,12/31/2013,WHITE HISPANIC,None,2010


In [40]:
#count amount of complaints by victim sex
sex_demographic = demographic.groupby(['VIC_SEX']).size().reset_index(name='complaint_count')

In [41]:
#drop columns which aren't used
sex_demographic1 = sex_demographic.drop([1,4])
sex_demographic1

,VIC_SEX,complaint_count
0,D,58505
2,F,195155
3,M,158775


In [42]:
#https://www.nyc.gov/assets/smallbizfirst/downloads/pdf/small-business-first-report.pdf, D bases on this report, where it is stated that there are 220000 businesses in new york city
#https://data.cityofnewyork.us/City-Government/Census-Demographics-at-the-NYC-City-Council-distri/ye4r-qpmp, male and female distribution based on this report
#actual nyc population data
sex_demographic1['SEX_DEM'] = [220000, 4214074, 3794204]

In [43]:
#create the expected distribution of complaints per category based on the population demographics
sex_demographic1['SEX_DEM_PER'] = sex_demographic1['SEX_DEM'] / sex_demographic1['SEX_DEM'].sum()
sex_demographic1['EXP_COM'] = sex_demographic1['SEX_DEM_PER'] * sex_demographic1['complaint_count'].sum()
sex_demographic1['EXP_COM'] = sex_demographic1['EXP_COM'].round(0)
sex_demographic1

,VIC_SEX,complaint_count,SEX_DEM,SEX_DEM_PER,EXP_COM
0,D,58505,220000,0.026737,11027.0
2,F,195155,4214074,0.512145,211227.0
3,M,158775,3794204,0.461118,190181.0


In [44]:
#create a table with only the relevant values
data2 = [sex_demographic1['complaint_count'],sex_demographic1['SEX_DEM']]

In [45]:
#chi square test
scipy.stats.chisquare(data2)


Power_divergenceResult(statistic=array([  93645.12315757, 3663159.68813618, 3343388.36964249]), pvalue=array([0., 0., 0.]))

In [46]:
#count amount of complaints by victim race
race_demographic = demographic.groupby(['VIC_RACE2']).size().reset_index(name='complaint_count')

In [47]:
race_demographic

,VIC_RACE2,complaint_count
0,AMERICAN INDIAN/ALASKAN NATIVE,2127
1,ASIAN / PACIFIC ISLANDER,25010
2,BLACK,117688
3,BLACK HISPANIC,15880
4,WHITE,88656
5,WHITE HISPANIC,78108


In [48]:
#https://slate.com/news-and-politics/2015/04/staten-island-for-sale-the-munsee-indians-sold-staten-island-under-duress-but-not-before-new-york-made-some-surprising-concessions.html, american indian/alaskan native from this article
#https://data.cityofnewyork.us/City-Government/Census-Demographics-at-the-NYC-City-Council-distri/ye4r-qpmp, other races based on this report
#https://furmancenter.org/files/sotc/The_Changing_Racial_and_Ethnic_Makeup_of_New_York_City_Neighborhoods_11.pdf, black/white hispanic composition based on this report
#actual race demographics in nyc
race_demographic['RACE_DEM'] = [111000, 783058, 1962154, 103707, 2801267, 198771
]

In [49]:
race_demographic

,VIC_RACE2,complaint_count,RACE_DEM
0,AMERICAN INDIAN/ALASKAN NATIVE,2127,111000
1,ASIAN / PACIFIC ISLANDER,25010,783058
2,BLACK,117688,1962154
3,BLACK HISPANIC,15880,103707
4,WHITE,88656,2801267
5,WHITE HISPANIC,78108,198771


In [50]:
#create the expected distribution of complaints per category based on the population demographics
race_demographic['RACE_DEM_PER'] = race_demographic['RACE_DEM'] / race_demographic['RACE_DEM'].sum()
race_demographic['EXP_COM'] = race_demographic['RACE_DEM_PER'] * race_demographic['complaint_count'].sum()
race_demographic['EXP_COM'] = race_demographic['EXP_COM'].round(0)

In [51]:
race_demographic

,VIC_RACE2,complaint_count,RACE_DEM,RACE_DEM_PER,EXP_COM
0,AMERICAN INDIAN/ALASKAN NATIVE,2127,111000,0.018624,6099.0
1,ASIAN / PACIFIC ISLANDER,25010,783058,0.131387,43025.0
2,BLACK,117688,1962154,0.329223,107810.0
3,BLACK HISPANIC,15880,103707,0.017401,5698.0
4,WHITE,88656,2801267,0.470015,153915.0
5,WHITE HISPANIC,78108,198771,0.033351,10921.0


In [52]:
#create a table with only the relevant values
data3 = [race_demographic['complaint_count'],race_demographic['EXP_COM']]

In [53]:
#chi square test
scipy.stats.chisquare(data3)


Power_divergenceResult(statistic=array([ 1917.91684902,  4770.19512016,   432.70842313,  4804.5752155 ,
       17556.66209481, 50703.62431343]), pvalue=array([0.00000000e+00, 0.00000000e+00, 4.18219458e-96, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00]))

In [54]:
#count amount of complaints by victim sex and drop unused column
age_demographic = demographic.groupby(['VIC_AGE_GROUP']).size().reset_index(name='complaint_count')
age_demographic = age_demographic.drop([5])

In [55]:
#actual age demographics in nyc
age_demographic['AGE_DEM'] = [803012, 2631301, 1695839, 937857, 1940269]

In [56]:
#create the expected distribution of complaints per category based on the population demographics
age_demographic['AGE_DEM_PER'] = age_demographic['AGE_DEM'] / age_demographic['AGE_DEM'].sum()
age_demographic['EXP_COM'] = age_demographic['AGE_DEM_PER'] * age_demographic['complaint_count'].sum()
age_demographic['EXP_COM'] = age_demographic['EXP_COM'].round(0)

In [57]:
#create a table with only the relevant values
data4 = [age_demographic['complaint_count'],age_demographic['AGE_DEM']]

In [58]:
age_demographic

,VIC_AGE_GROUP,complaint_count,AGE_DEM,AGE_DEM_PER,EXP_COM
0,18-24,53441,803012,0.100273,33498.0
1,25-44,159208,2631301,0.328573,109767.0
2,45-64,80799,1695839,0.211761,70744.0
3,65+,15072,937857,0.117111,39124.0
4,<18,25553,1940269,0.242283,80940.0


In [59]:
#chi square test
scipy.stats.chisquare(data3)


Power_divergenceResult(statistic=array([ 1917.91684902,  4770.19512016,   432.70842313,  4804.5752155 ,
       17556.66209481, 50703.62431343]), pvalue=array([0.00000000e+00, 0.00000000e+00, 4.18219458e-96, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00]))

In [60]:
#only look at the data where vic race is unknown
data5 =data.loc[data['VIC_RACE'] == 'UNKNOWN']

In [61]:
data

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,OCC,OCC_END,VIC_RACE2,VIC_AGE_GROUP2,Year
0,506547392,03/29/2018,20:30:00,<NA>,NaN,32.0,2018-03-30,351,CRIMINAL MISCHIEF & RELATED OF,254.0,...,PATROL BORO MAN NORTH,NaN,25-44,WHITE,F,2018-03-29,<NA>,WHITE,None,2018
1,629632833,02/06/2018,23:15:00,<NA>,NaN,52.0,2018-02-07,341,PETIT LARCENY,333.0,...,PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,D,2018-02-06,<NA>,None,45-64,2018
2,787203902,11/21/2018,00:15:00,11/21/2018,00:20:00,75.0,2018-11-21,341,PETIT LARCENY,321.0,...,PATROL BORO BKLYN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-11-21,11/21/2018,None,25-44,2018
3,280364018,06/09/2018,21:42:00,06/09/2018,21:43:00,10.0,2018-06-10,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,...,PATROL BORO MAN SOUTH,NaN,18-24,WHITE HISPANIC,F,2018-06-09,06/09/2018,WHITE HISPANIC,25-44,2018
4,985800320,11/10/2018,19:40:00,11/10/2018,19:45:00,19.0,2018-11-10,341,PETIT LARCENY,333.0,...,PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,D,2018-11-10,11/10/2018,None,<18,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7825494,104527061,05/01/2008,18:55:00,05/01/2018,19:00:00,10.0,2018-05-01,235,DANGEROUS DRUGS,511.0,...,PATROL BORO MAN SOUTH,NaN,UNKNOWN,UNKNOWN,E,2008-05-01,05/01/2018,None,45-64,2008
7825495,284201488,05/18/2018,14:50:00,05/18/2018,14:55:00,122.0,2018-05-22,344,ASSAULT 3 & RELATED OFFENSES,101.0,...,PATROL BORO STATEN ISLAND,NaN,<18,WHITE HISPANIC,M,2018-05-18,05/18/2018,WHITE HISPANIC,<18,2018
7825496,808565901,11/19/2018,02:25:00,11/19/2018,02:30:00,110.0,2018-11-19,109,GRAND LARCENY,421.0,...,PATROL BORO QUEENS NORTH,NaN,25-44,ASIAN / PACIFIC ISLANDER,M,2018-11-19,11/19/2018,ASIAN / PACIFIC ISLANDER,None,2018
7825497,649441648,02/03/2018,10:02:00,02/03/2018,12:15:00,122.0,2018-02-03,578,HARRASSMENT 2,638.0,...,PATROL BORO STATEN ISLAND,NaN,45-64,WHITE,M,2018-02-03,02/03/2018,WHITE,45-64,2018


In [64]:
#look at the sex of the victims whose race is unknown
data5['VIC_SEX'].value_counts()

E    1183709
D     965655
F     203513
M     200236
U          3
Name: VIC_SEX, dtype: int64